<a href="https://colab.research.google.com/github/LuisFerRosas/transcripcion_de_Musica/blob/main/nuevo__modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/Proyecto_ia3

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/Proyecto_ia3


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch import Tensor 
import math
import json

In [ ]:
!ls

 checkpoint			    'Final Transformer.ipynb'
'Copia de Final Transformer.ipynb'  'nuevo_ modelo.ipynb'
 datos2				     transcripcion_de_Musica


In [ ]:
with open("./datos2/vocabulario2.json", "r") as fp:
        vocabPart = json.load(fp)

len(vocabPart)

233

In [ ]:
!git clone https://github.com/LuisFerRosas/transcripcion_de_Musica.git

fatal: destination path 'transcripcion_de_Musica' already exists and is not an empty directory.


In [ ]:
cd transcripcion_de_Musica/

/gdrive/My Drive/Colab Notebooks/Proyecto_ia3/transcripcion_de_Musica


In [ ]:
!pip install -r requirements.txt

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class PositionalEncoding(nn.Module):
  def __init__(self,
                emb_size: int,
                dropout: float,
                maxlen: int = 6000):
      super(PositionalEncoding, self).__init__()
      den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
      pos = torch.arange(0, maxlen).reshape(maxlen, 1)
      pos_embedding = torch.zeros((maxlen, emb_size))
      pos_embedding[:, 0::2] = torch.sin(pos * den)
      pos_embedding[:, 1::2] = torch.cos(pos * den)
      pos_embedding = pos_embedding.unsqueeze(-2)

      self.dropout = nn.Dropout(dropout)
      self.register_buffer('pos_embedding', pos_embedding)

  def forward(self, token_embedding: Tensor):
      return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [ ]:
class TmusicTrasforms(nn.Module):
  def __init__(self,tgt_vocabulario):
    super().__init__() 
    
    ################ TRANSFORMER BLOCK #############################
    # maxpool the input feature map/tensor to the transformer 
    # a rectangular kernel worked better here for the rectangular input spectrogram feature map/tensor
    self.transformer_maxpool = nn.MaxPool2d(kernel_size=[1,4], stride=[1,4])
    
    
    self.tgt_tok_emb = TokenEmbedding(tgt_vocabulario, 512)
    self.positional_encoding = PositionalEncoding(512, dropout=0.3)
    
    transformer_layer = nn.TransformerEncoderLayer(
        d_model=40, # input feature (frequency) dim after maxpooling 40*282 -> 40*70 (MFC*time)
        nhead=4, # 4 self-attention layers in each multi-head self-attention layer in each encoder block
        dim_feedforward=512, # 2 linear layers in each encoder block's feedforward network: dim 40-->512--->40
        dropout=0.4, 
        activation='relu' # ReLU: avoid saturation/tame gradient/reduce compute time
    )
    
    self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=4)
    
    transformer_decoder_layer=nn.TransformerDecoderLayer(
        d_model=512,
        nhead=4,
        dim_feedforward=512,
        dropout=0.4,
        activation='relu'
    )
    self.transformer_decoder=nn.TransformerDecoder(transformer_decoder_layer,num_layers=6)

    ############### 1ST PARALLEL 2D CONVOLUTION BLOCK ############
    # 3 sequential conv2D layers: (1,40,282) --> (16, 20, 141) -> (32, 5, 35) -> (64, 1, 8)
    self.conv2Dblock1 = nn.Sequential(
        
        # 1st 2D convolution layer
        nn.Conv2d(
            in_channels=1, # input volume depth == input channel dim == 1
            out_channels=8, # expand output feature map volume's depth to 16
            kernel_size=3, # typical 3*3 stride 1 kernel
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(8), # batch normalize the output feature map before activation
        nn.ReLU(), # feature map --> activation map
        nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
        nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
        
        # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
        nn.Conv2d(
            in_channels=8, 
            out_channels=16, # expand output feature map volume's depth to 32
            kernel_size=3,
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=5, stride=5), # increase maxpool kernel for subsequent filters
        nn.Dropout(p=0.3), 

        # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
        nn.Conv2d(
            in_channels=16, 
            out_channels=32, # expand output feature map volume's depth to 32
            kernel_size=3,
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # increase maxpool kernel for subsequent filters
        nn.Dropout(p=0.3), 
        
        # 3rd 2D convolution layer identical to last except output dim
        nn.Conv2d(
            in_channels=32,
            out_channels=64, # expand output feature map volume's depth to 64
            kernel_size=3,
            stride=1,
            padding=1,
            
                  ),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(p=0.3),
    )
    ############### 2ND PARALLEL 2D CONVOLUTION BLOCK ############
    # 3 sequential conv2D layers: (1,40,282) --> (16, 20, 141) -> (32, 5, 35) -> (64, 1, 8)
    self.conv2Dblock2 = nn.Sequential(
      # 1st 2D convolution layer
        nn.Conv2d(
            in_channels=1, # input volume depth == input channel dim == 1
            out_channels=8, # expand output feature map volume's depth to 16
            kernel_size=3, # typical 3*3 stride 1 kernel
            stride=1,
            padding=1
                  ),
        nn.BatchNorm2d(8), # batch normalize the output feature map before activation
        nn.ReLU(), # feature map --> activation map
        nn.MaxPool2d(kernel_size=2, stride=2), #typical maxpool kernel size
        nn.Dropout(p=0.3), #randomly zero 30% of 1st layer's output feature map in training
        
        # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
        nn.Conv2d(
            in_channels=8, 
            out_channels=16, # expand output feature map volume's depth to 32
            kernel_size=3,
            stride=1,
            padding=1
                  ),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=5, stride=5), # increase maxpool kernel for subsequent filters
        nn.Dropout(p=0.3), 

        # 2nd 2D convolution layer identical to last except output dim, maxpool kernel
        nn.Conv2d(
            in_channels=16, 
            out_channels=32, # expand output feature map volume's depth to 32
            kernel_size=3,
            stride=1,
            padding=1
                  ),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2), # increase maxpool kernel for subsequent filters
        nn.Dropout(p=0.3), 
        
        # 3rd 2D convolution layer identical to last except output dim
        nn.Conv2d(
            in_channels=32,
            out_channels=64, # expand output feature map volume's depth to 64
            kernel_size=3,
            stride=1,
            padding=1
                  ),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(p=0.3),
    
    )

    ################# FINAL LINEAR BLOCK ####################
    # Linear softmax layer to take final concatenated embedding tensor 
    #    from parallel 2D convolutional and transformer blocks, output 8 logits 
    # Each full convolution block outputs (64*1*8) embedding flattened to dim 512 1D array 
    # Full transformer block outputs 40*70 feature map, which we time-avg to dim 40 1D array
    # 512*2+40 == 1064 input features --> 8 output emotions 
    
    self.fc2_linear = nn.Linear(512,tgt_vocabulario)
    
    ### Softmax layer for the 8 output logits from final FC linear layer 
    self.softmax_out = nn.Softmax(dim=1) # dim==1 is the freq embedding
    
    # define one complete parallel fwd pass of input feature tensor thru 2*conv+1*transformer blocks
  def forward(self,x,partitura_tok):
    
    ############ 1st parallel Conv2D block: 4 Convolutional layers ############################
    # create final feature embedding from 1st convolutional layer 
    # input features pased through 4 sequential 2D convolutional layers
    conv2d_embedding1 = self.conv2Dblock1(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding1 : "+str(conv2d_embedding1.shape))
    # flatten final 64*1*8 feature map from convolutional layers to length 512 1D array 
    # skip the 1st (N/batch) dimension when flattening
    conv2d_embedding1 = torch.flatten(conv2d_embedding1, start_dim=1) 
    # print("conv2d_embedding1 flatten : "+str(conv2d_embedding1.shape))
    ############ 2nd parallel Conv2D block: 4 Convolutional layers #############################
    # create final feature embedding from 2nd convolutional layer 
    # input features pased through 4 sequential 2D convolutional layers
    conv2d_embedding2 = self.conv2Dblock2(x) # x == N/batch * channel * freq * time
    # print("conv2d_embedding2 : "+str(conv2d_embedding2.shape))
    # flatten final 64*1*8 feature map from convolutional layers to length 512 1D array 
    # skip the 1st (N/batch) dimension when flattening
    conv2d_embedding2 = torch.flatten(conv2d_embedding2, start_dim=1) 
    # print("conv2d_embedding2 flatten : "+str(conv2d_embedding2.shape))
      
    ########## 4-encoder-layer Transformer block w/ 40-->512-->40 feedfwd network ##############
    # maxpool input feature map: 1*40*282 w/ 1*4 kernel --> 1*40*70 
    x_maxpool = self.transformer_maxpool(x)
    # print("x_maxpool : "+str(x_maxpool.shape))
    # remove channel dim: 1*40*70 --> 40*70
    x_maxpool_reduced = torch.squeeze(x_maxpool,1)
    # print("x_maxpool_reduced : "+str(x_maxpool_reduced.shape))
    # convert maxpooled feature map format: batch * freq * time ---> time * batch * freq format
    # because transformer encoder layer requires tensor in format: time * batch * embedding (freq)
    x = x_maxpool_reduced.permute(2,0,1) 
    # print("x------>entrada transformer : "+str(x.shape))
    # finally, pass reduced input feature map x into transformer encoder layers
    transformer_output = self.transformer_encoder(x)
    # print("salida transformer : "+str(transformer_output.shape))
    
    # create final feature emedding from transformer layer by taking mean in the time dimension (now the 0th dim)
    # transformer outputs 2x40 (MFCC embedding*time) feature map, take mean of columns i.e. take time average
    transformer_embedding = torch.mean(transformer_output, dim=0) # dim 40x70 --> 40
    # print("transformer_embedding media : "+str(transformer_embedding.shape))
  
    complete_embedding = torch.cat([conv2d_embedding1, conv2d_embedding2,transformer_embedding], dim=1)  
    # print("complete_embedding  : "+str(complete_embedding.shape))
    cambDim = complete_embedding.unsqueeze(2)
    # print("cambDim  : "+str(cambDim.shape))

    expandido=cambDim.expand([2,5544,512])
    # print("expandido  : "+str(expandido.shape))

    memory=expandido.transpose(1,0)
    # print("memory  : "+str(memory.shape))
      
    partitura_tok=partitura_tok.transpose(1,0)
    # print("partitura_tok  : "+str(partitura_tok.shape))
    partitura_tok= self.tgt_tok_emb(partitura_tok)
    # print("partitura_tok2  : "+str(partitura_tok.shape))
    tgt_emb = self.positional_encoding(partitura_tok)
    # print("tgt_emb_positional  : "+str(tgt_emb.shape))

    mask_tgt=generate_square_subsequent_mask(tgt_emb.shape[0])
    ouput_decoder=self.transformer_decoder(tgt_emb,memory,mask_tgt)
    # print("ouput_decoder  : "+str(ouput_decoder.shape))
    linear_decoder=self.fc2_linear(ouput_decoder)
    # print("linear_decoder  : "+str(linear_decoder.shape))
   
   
    ######### Final Softmax layer: use logits from FC linear, get softmax for prediction ######
    output_softmax = self.softmax_out(linear_decoder)
    # print("output_softmax  : "+str(output_softmax.shape))
    # need output logits to compute cross entropy loss, need softmax probabilities to predict class
    return  output_softmax 
  
  

def generate_square_subsequent_mask(sz):
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


In [ ]:
from sklearn.metrics import precision_score
def make_train_step(model, criterion, optimizer):
    
  def train_step(espectro,partituraT,lenPartituras):
    model.train() 
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    output_softmax = model(espectro,partituraT)
    salida =output_softmax.transpose(0,1)
   
   
    sumAcuraccy=0
    for i in range(salida.shape[0]):
      predictions=torch.argmax(salida[i],dim=1)
      pred=predictions.detach().cpu().numpy()
      parti=partituraT[i].detach().cpu().numpy()
      acuraccy=precision_score(pred,parti,average='micro')
      
      # print("acuraccy :: "+str(acuraccy))
      
      sumAcuraccy+=acuraccy

    sumAcuraccy=sumAcuraccy/salida.shape[0]
    
    output_lengths = torch.full((output_softmax.shape[1],), output_softmax.shape[0], dtype=torch.long)
    output_lengths =output_lengths.to(device)
    loss = criterion(output_softmax,partituraT,output_lengths,lenPartituras) 
    
    # compute gradients for the optimizer to use 
    loss.backward()
    
    # update network parameters based on gradient stored (by calling loss.backward())
    optimizer.step()
    
    # zero out gradients for next pass
    # pytorch accumulates gradients from backwards passes (convenient for RNNs)
    optimizer.zero_grad() 
    
    return loss.item(),sumAcuraccy
  return train_step

In [ ]:
def make_validate_fnc(model,criterion):
  def validate(espectro,partituraT,lenPartituras):
      
    # don't want to update any network parameters on validation passes: don't need gradient
    # wrap in torch.no_grad to save memory and compute in validation phase: 
    with torch.no_grad(): 
        
      # set model to validation phase i.e. turn off dropout and batchnorm layers 
      model.eval()

      # get the model's predictions on the validation set
      output_softmax = model(espectro,partituraT)
      salida =output_softmax.transpose(0,1)
      sumAcuraccy=0
      for i in range(salida.shape[0]):
        predictions=torch.argmax(salida[i],dim=1)
        pred=predictions.detach().cpu().numpy()
        parti=partituraT[i].detach().cpu().numpy()
        acuraccy=precision_score(pred,parti,average='micro')
        # print("acuraccy :: "+str(acuraccy))      
        sumAcuraccy+=acuraccy

    sumAcuraccy=sumAcuraccy/salida.shape[0]
    
    output_lengths = torch.full((output_softmax.shape[1],), output_softmax.shape[0], dtype=torch.long)
    
    loss = criterion(output_softmax,partituraT,output_lengths,lenPartituras) 
        
    return loss.item(),sumAcuraccy
  return validate

In [ ]:
def make_save_checkpoint(): 
  def save_checkpoint(optimizer, model, epoch, filename):
    checkpoint_dict = {
        'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint_dict, filename)
  return save_checkpoint

def load_checkpoint(optimizer, model, filename):
  checkpoint_dict = torch.load(filename)
  epoch = checkpoint_dict['epoch']
  model.load_state_dict(checkpoint_dict['model'])
  if optimizer is not None:
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
  return epoch

In [ ]:
#hiperparametros
batch_size=2
inicio_EPOCH=0
final_EPOCH=100
tgt_vocabulario=len(vocabPart)

In [ ]:
from preprocess import collate_fn_nuevo, obtenerDatos
from tensorboardX import SummaryWriter
from tqdm import tqdm
from torch.utils.data import  DataLoader

In [ ]:




# set device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} selected')

# instantiate model and move to GPU for training
model=TmusicTrasforms(tgt_vocabulario=tgt_vocabulario)
model=model.to(device)
print('Numero de parametros entrenable: ',sum(p.numel() for p in model.parameters()) )


optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

criterion = nn.CTCLoss()
criterion=criterion.to(device)

save_checkpoint = make_save_checkpoint()


train_step = make_train_step(model, criterion, optimizer=optimizer)


validate = make_validate_fnc(model,criterion)



def train(num_epochs,model):
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  dataset_train =obtenerDatos(pathPartituras='../datos2/partituras',pathVocabulario='../datos2/vocabulario2.json',
                          pathArchivoNPY='../datos2/archivosnumpy/train_audio_nombre.npy')
    
  dataset_valid =obtenerDatos(pathPartituras='../datos2/partituras',pathVocabulario='../datos2/vocabulario2.json',
                          pathArchivoNPY='../datos2/archivosnumpy/valid_audio_nombre.npy')
  writer = SummaryWriter("runs/transformer")
  dataloader_train = DataLoader(dataset_train, batch_size=1, collate_fn=collate_fn_nuevo ,shuffle=True)
  dataloader_valid = DataLoader(dataset_valid, batch_size=1, collate_fn=collate_fn_nuevo ,shuffle=True)
  estep=0
  for epoch in range(num_epochs):      
    estep+=1
    pbar_train = tqdm(dataloader_train)
    
    loss_train=[]
    acuraccy_train=[]
    loss_valid=[]
    acuraccy_valid=[]
    for i, data in enumerate(pbar_train):
      partitura_tokenizada,wave_mfcc,len_partitura = data
      # print(wave_mfcc.type())
      partitura_tokenizada = partitura_tokenizada.to(device)
      # wave_mfcc=wave_mfcc.float()
      wave_mfcc = wave_mfcc.to(device)
     
      len_partitura= len_partitura.to(device)
      loss,acuraccy= train_step(wave_mfcc,partitura_tokenizada,len_partitura)
      loss_train.append(loss)
      acuraccy_train.append(acuraccy)
      if estep==1:
        writer.add_graph(model ,input_to_model=[wave_mfcc,partitura_tokenizada])

    for data2 in dataloader_valid:
      partitura_tokenizada2,wave_mfcc2,len_partitura2 = data2
      partitura_tokenizada2=partitura_tokenizada2.to(device)
      # wave_mfcc=wave_mfcc.float()
      wave_mfcc2=wave_mfcc2.to(device)
      
      len_partitura2=len_partitura2.to(device)
      loss_val2,acuraccy_val2=validate(wave_mfcc2,partitura_tokenizada2,len_partitura2)
      loss_valid.append(loss_val2)
      acuraccy_valid.append(acuraccy_val2)

    final_loss_train=sum(loss_train)/len(loss_train)
    final_acuraccy_train=sum(acuraccy_train)/len(acuraccy_train)
    final_loss_valid=sum(loss_valid)/len(loss_valid)
    final_acuraccy_valid=sum(acuraccy_valid)/len(acuraccy_valid)
    writer.add_scalar("loss_train :",final_loss_train ,epoch)
    writer.add_scalar("acuraccy_train :",final_acuraccy_train ,epoch)
    writer.add_scalar("loss_valid :",final_loss_valid ,epoch)
    writer.add_scalar("acuraccy_valid :",final_acuraccy_valid ,epoch)

    pbar_train.set_description("Procesando la epoca %d"%epoch)
    print(f'loss_train: {final_loss_train}, loss_valid: {final_loss_valid} , acuraccy_train: {final_acuraccy_train}, acuraccy_valid: {final_acuraccy_valid}')
      
    # checkpoint_filename = '/content/gdrive/My Drive/DL/models/checkpoints/parallel_all_you_wantFINAL-{:03d}.pkl'.format(epoch)
    # save_checkpoint(optimizer, model, epoch, checkpoint_filename)


cuda:0 selected
Numero de parametros entrenable:  16258793


In [ ]:



for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
train(5,model)

  0%|          | 0/300 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
 88%|████████▊ | 265/300 [09:13<01:16,  2.19s/it]

In [ ]:
import torch
inputt = torch.randn(2,1, 40,1723, requires_grad=True)
partitura = torch.randint( 0,132, (2,100))
pato = torch.randn(2,5544, requires_grad=True)
lenParitura = torch.randint( 30,50, (2,))
prueba = torch.randn(100,2,512, requires_grad=True)
print(inputt.shape)
print(partitura.shape)
print(lenParitura)
print(prueba.shape[0])

In [ ]:
model=TmusicTrasforms(tgt_vocabulario=233)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model=model.to(DEVICE)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
criterion = nn.CTCLoss()

In [ ]:
train_step=make_train_step(model,criterion,optimizer)
loss=train_step(inputt,partitura,lenParitura)
loss

In [ ]:
model.train()
ouput=model(inputt,partitura)
ouput

In [ ]:
from torchsummary import summary

# need device to instantiate model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# instantiate model for 8 emotions and move to GPU 
model = TmusicTrasforms(8).to(device)

# include input feature map dims in call to summary()
summary(model, input_size=[(1,40,1723),(1,100)])


In [ ]:
alto=torch.max(pato)
alto

In [ ]:
emnb =nn.Embedding(133,512)

salida=emnb(partitura.long())
salida.shape

In [ ]:
pato[0]

In [ ]:
pato.shape

In [ ]:
x = pato.unsqueeze(2)
print(x.size())

x=x.expand([2,5544,512])
print(x.size())

In [ ]:
x[0]

In [ ]:
ys = torch.ones(1, 1).fill_(1).type(torch.long)
ys.shape

In [ ]:
ys = torch.cat([ys,torch.ones(1, 1).type_as().fill_(45)], dim=0)

In [ ]:
log_probs = torch.randn(100, 2, 233).log_softmax(2).detach().requires_grad_()
print(log_probs.shape)#salida del modelo
targets = torch.randint(0, 233, (2, 100), dtype=torch.long)
print(targets.shape)#tokens del las partituras
input_lengths = torch.full((2,), 50, dtype=torch.long)
print(input_lengths)#salida de 100 caracteristicas salida decoder
target_lengths = torch.randint(10,30,(2,), dtype=torch.long)
print(target_lengths)#tamanio de las partituras

In [ ]:
log_probs.shape[1]

In [ ]:
import torch.nn.functional as F  
criterion =nn.CTCLoss()
loss=criterion(log_probs, targets, input_lengths, target_lengths)
loss